In [4]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('../dataset/dataset.csv')

# ---------------------------
# 1. Data Loading & Parsing
# ---------------------------
def parse_signal(signal_str):
    """Convert complex string to magnitude/phase features"""
    cleaned = signal_str.strip('[]').replace(' ', '')
    parts = cleaned.split('),(')
    signal = []
    for p in parts:
        p = p.replace('(', '').replace(')', '')
        try:
            cmplx = complex(p)
            signal.append([abs(cmplx), np.angle(cmplx)])  # Magnitude + Phase
        except ValueError:
            continue
    return np.array(signal[:208])  # Shape: (208, 2)

def parse_secret_code(code_str):
    """Convert secret code string to integer array"""
    return np.array([int(x) for x in code_str.strip('[]').split(', ')])


# Parse all columns
X_signal = np.array([parse_signal(s) for s in df['received_signal']])  # (15000, 208, 2)
X_secret = np.array([parse_secret_code(c) for c in df['secret_code']])  # (15000, 13)
X_secret_shifted = X_secret + 1  # Now values are 0-1000
input_dim = np.max(X_secret_shifted) + 1  # 1001 for your data
y = df[['jet1_x', 'jet1_y', 'jet1_z', 'jet2_x', 'jet2_y', 'jet2_z']].values  # (15000, 6)

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ---------------------------
# 2. Data Splitting & Normalization
# ---------------------------
# First split: 80% train, 20% temp
X_sig_train_raw, X_sig_temp_raw, X_sec_train_raw, X_sec_temp_raw, y_train_raw, y_temp_raw = train_test_split(
    X_signal, X_secret_shifted , y, test_size=0.2, random_state=42
)

# Second split: 50% validation, 50% test
X_sig_val_raw, X_sig_test_raw, X_sec_val_raw, X_sec_test_raw, y_val_raw, y_test_raw = train_test_split(
    X_sig_temp_raw, X_sec_temp_raw, y_temp_raw, test_size=0.5, random_state=42
)

# Signal Normalizer (fit on training only)
scaler_signal = StandardScaler()
X_sig_train_flat = X_sig_train_raw.reshape(-1, 2)
scaler_signal.fit(X_sig_train_flat)

def scale_and_reshape(X_raw, scaler):
    X_flat = X_raw.reshape(-1, 2)
    X_scaled = scaler.transform(X_flat)
    return X_scaled.reshape(-1, 208, 2)

X_sig_train = scale_and_reshape(X_sig_train_raw, scaler_signal)
X_sig_val = scale_and_reshape(X_sig_val_raw, scaler_signal)
X_sig_test = scale_and_reshape(X_sig_test_raw, scaler_signal)

# Secret Code Normalizer
scaler_secret = StandardScaler()
scaler_secret.fit(X_sec_train_raw)

X_sec_train = scaler_secret.transform(X_sec_train_raw)
X_sec_val = scaler_secret.transform(X_sec_val_raw)
X_sec_test = scaler_secret.transform(X_sec_test_raw)

# Target Normalizer
scaler_target = StandardScaler()
scaler_target.fit(y_train_raw)

y_train = scaler_target.transform(y_train_raw)
y_val = scaler_target.transform(y_val_raw)
y_test = scaler_target.transform(y_test_raw)


# --------------------------------------------------
# 4. Verification
# --------------------------------------------------
print("Training shapes:")
print(f"Signals: {X_sig_train.shape}, Codes: {X_sec_train.shape}, Targets: {y_train.shape}")
print("\nValidation shapes:")
print(f"Signals: {X_sig_val.shape}, Codes: {X_sec_val.shape}, Targets: {y_val.shape}")
print("\nTest shapes:")
print(f"Signals: {X_sig_test.shape}, Codes: {X_sec_test.shape}, Targets: {y_test.shape}")

Training shapes:
Signals: (12000, 208, 2), Codes: (12000, 13), Targets: (12000, 6)

Validation shapes:
Signals: (1500, 208, 2), Codes: (1500, 13), Targets: (1500, 6)

Test shapes:
Signals: (1500, 208, 2), Codes: (1500, 13), Targets: (1500, 6)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.layers import BatchNormalization, GlobalMaxPooling1D, Dropout, Embedding, Dot, Softmax
from tensorflow.keras import layers
import tensorflow as tf
import math


# --------------------------------------------------
# 3.1 Dual Input Branches (Upgraded)
# --------------------------------------------------
# Branch 1: Radar Signal Processor
signal_input = Input(shape=(208, 2))
x = Conv1D(256, 5, activation='relu', padding='same')(signal_input)
x = BatchNormalization()(x)
x = MaxPooling1D(2)(x)
x = Conv1D(512, 3, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(2)(x)
x = Conv1D(1024, 3, activation='relu', padding='same')(x)
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.4)(x)
# this was not at all good for the 15000 samples model

# Branch 2: Code Breaker (With Shifted Codes)
code_input = Input(shape=(13,))
y = Embedding(input_dim=1001, output_dim=16)(code_input)  # Fixed input_dim
y = Flatten()(y)
y = Dense(512, activation='relu')(y)
y = Dense(256, activation='relu')(y)
y = BatchNormalization()(y)
y = Dropout(0.3)(y)

# --------------------------------------------------
# 3.2 Enhanced Fusion
# --------------------------------------------------
merged = concatenate([x, y])
z = Dense(1024, activation='relu')(merged)
z = Dense(512, activation='relu')(z)
z = Dense(256, activation='relu')(z)
outputs = Dense(6, activation='linear')(z)

# Model Compilation
model = Model(inputs=[signal_input, code_input], outputs=outputs)
optimizer = Adam(learning_rate=0.001)

def weighted_mse(y_true, y_pred):
    # Assign higher weights to x, y, z coordinates for both jets
    weights = tf.constant([1.2, 1.2, 1.2, 1.2, 1.2, 1.2], dtype=tf.float32)
    squared_diff = tf.square(y_true - y_pred)
    weighted_squared_diff = squared_diff * weights
    return tf.reduce_mean(weighted_squared_diff)

model.compile(
    optimizer=optimizer,
    loss=weighted_mse,
    metrics=['mae']
)

# --------------------------------------------------
# Reconnaissance Report (Model Summary)
# --------------------------------------------------
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 208, 2)]             0         []                            
                                                                                                  
 conv1d_3 (Conv1D)           (None, 208, 256)             2816      ['input_3[0][0]']             
                                                                                                  
 batch_normalization_3 (Bat  (None, 208, 256)             1024      ['conv1d_3[0][0]']            
 chNormalization)                                                                                 
                                                                                                  
 max_pooling1d_2 (MaxPoolin  (None, 104, 256)             0         ['batch_normalization_3[

In [10]:
optimizer = Adam(
    learning_rate=0.0001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
)

early_stop = EarlyStopping(
    monitor='val_mae',
    patience=20,
    mode='min',
    restore_best_weights=True
)

# Learning Rate Scheduler
def lr_scheduler(epoch):
    if epoch < 20:
        return 0.0001
    elif epoch < 50:
        return 0.00005
    else:
        return 0.00001

lr_callback = LearningRateScheduler(lr_scheduler)

In [11]:
history = model.fit(
    [X_sig_train, X_sec_train],  # Use shifted codes
    y_train,
    validation_data=([X_sig_val, X_sec_val], y_val),
    epochs=50,
    batch_size=32,  # Reduced for better convergence
    callbacks=[early_stop, lr_callback],
    verbose=1
)

Epoch 1/50
375/375 [==============================] - 165s 437ms/step - loss: 0.9386 - mae: 0.7253 - val_loss: 0.8417 - val_mae: 0.6832 - lr: 1.0000e-04
Epoch 2/50
375/375 [==============================] - 166s 442ms/step - loss: 0.7384 - mae: 0.6319 - val_loss: 0.6739 - val_mae: 0.6006 - lr: 1.0000e-04
Epoch 3/50
375/375 [==============================] - 153s 407ms/step - loss: 0.7056 - mae: 0.6153 - val_loss: 0.6728 - val_mae: 0.6017 - lr: 1.0000e-04
Epoch 4/50
375/375 [==============================] - 151s 404ms/step - loss: 0.6857 - mae: 0.6074 - val_loss: 0.6704 - val_mae: 0.6004 - lr: 1.0000e-04
Epoch 5/50
375/375 [==============================] - 173s 460ms/step - loss: 0.6615 - mae: 0.5969 - val_loss: 0.6855 - val_mae: 0.6061 - lr: 1.0000e-04
Epoch 6/50
375/375 [==============================] - 170s 453ms/step - loss: 0.6242 - mae: 0.5781 - val_loss: 0.6879 - val_mae: 0.6033 - lr: 1.0000e-04
Epoch 7/50
375/375 [==============================] - 174s 464ms/step - loss: 0.55

KeyboardInterrupt: 

In [12]:
test_loss, test_mae = model.evaluate(  
    [X_sig_test, X_sec_test],   
    y_test,  
    verbose=0  
)  
print(f"Test MSE: {test_loss:.4f} (Normalized)")  
print(f"Test MAE: {test_mae:.4f} (Normalized)")  

# Inverse-transform for real-world error  
y_pred_normalized = model.predict([X_sig_test, X_sec_test])  
y_pred_real = scaler_target.inverse_transform(y_pred_normalized)  
y_test_real = scaler_target.inverse_transform(y_test)  

# Calculate real-world MAE  
mae_real = np.mean(np.abs(y_pred_real - y_test_real))  
print(f"Real-World MAE: {mae_real:.2f} meters")  

Test MSE: 0.8911 (Normalized)
Test MAE: 0.6348 (Normalized)
47/47 [==============================] - 5s 93ms/step
Real-World MAE: 5946.05 meters
